# Introduction

I went through several steps while building the game. Each section build on the previous one:

Stage 1: person vs person game that just lets two players play through the console

Stage 2: person vs bot game where the bot picks random moves

Stage 3: Smart bot where the bot uses minimax to make decisions

Stage 4: Smarter bot where the minimax algorithm also has alpha beta pruning

Stage 5: Smarter bot where the bot also uses an evaluation function

## Stage 1: Person vs Person

In [94]:
class TicTacToeGame:
    """
    Class TicTacToeGame: Provides a structure for playing a Tic-Tac-Toe Game
    
    Attributes
    ----------
        board_dimension: int
            The dimension of the board
        board: list of lists
            Used to represent the board. The options are '_' for empty square, 'X', or 'O'
            
    Methods
    -------
        Dunder method str is overwritten.

        print_board: string
            Prints the board in a nicely formatted way
        create_board: list of lists
            Used to initialize the empty board at the beginning of the game
        is_board_full: Bool
            Checks whetehr there are any empty squares left
        check_winner: int
            Checks the board to see if the confuguration is winning. Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player. 
        play_game:
            Starts a tic tac toe game. 
    """

    # Class constructor
    def __init__(self, board_dimension):
        self.board_demension = board_dimension
        self.board = self.create_board()

    def create_board(self):
        """ Method to initialize the empty board in the beginning of the game."""
        return [['_' for i in range(self.board_demension)] for j in range(self.board_demension)]

    # Convert to string. 
    def __str__(self):
        output = ('\n'.join(map(str, self.board)))
        return output

    def print_board(self):
        """Print the board configuration. Used inside other methods (since I can't call __str__ directly"""
        output = ('\n'.join(map(str, self.board)))
        return output

    def is_board_full(self):
        """A function to check if there are any possible moves left (whether the board is filled out or there are any empty squares left)"""
        for i in range(self.board_demension):
            for j in range(self.board_demension):
                if self.board[i][j] == '_':
                    return False
        return True

    def check_winner(self, player):
        """
        Checks whether the current configuration of the board is winning. Acts as a simple evaluation function: 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player.

        Parameters
        __________
        player: Player class instance
        
        Return
        ______
        int: 
            Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player. 
        """
        
        # Check for winning row. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # print("row", i, "column", j) ### DELETE PRINTS USED FOR DEBUGGING
                # Update truth value: if the squares match and they are not empty. 
                condition = condition and (self.board[i][j] == self.board[i][j-1]) and (self.board[i][j] != '_')
                # print("Comparing", self.board[i][j], "and", self.board[i][j-1])
                # print("New condition value is:", condition)
                # If row values don't match, stop checking this row
                if  not condition: 
                    break
            
            # print("CONDITION RIGHT BEFORE IF STATEMENT", condition)
            # If all entries in the row are matching return the value. 
            if condition:
                # print("CONDITION IN IF STATEMENT", condition)
                if player.maximizing: return 1
                else: return -1

        # Check for winning column. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # Update truth value. 
                condition = condition and (self.board[j][i] == self.board[j-1][i])  and (self.board[j][i] != '_')
                # If row values don't match, stop checking this row
                if  not condition: break
            
            # If all entries in the row are matching return the value. 
            if condition:
                if player.maximizing: return 1
                else: return -1

        # Check for winning diagonal (left to right). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][i] == self.board[i-1][i-1]) and self.board[i][i] != "_"
                if not diagonal_condition: break
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            if player.maximizing: return 1
            else: return -1

         # Check for winning diagonal (right to left). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][0-(i+1)] == self.board[i-1][0-i]) and self.board[i-1][0-i] != "_"

            if not diagonal_condition: break
            
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            if player.maximizing: return 1
            else: return -1

        # No winner, it's a draw.
        return 0 

    def play_game(self, player1, player2):
        """
        Starts a tic- tac-toe game
        
        Parameters
        _________
        player1: Player class instance
        player2: Player class instance

        Return
        ______
        None
        """

        player = player1

        while True:
            print(f"Player {player.symbol}'s turn")

            print(self.print_board())

            # Approach taken from https://geekflare.com/tic-tac-toe-python-code/ but adapted in my own way. 
            # Get player's turn
            while True:
                row, column = list(
                    map(int, input("Enter row and column numbers to place your symbol (ON AN EMPTY SQUARE): ").split()))
                print()
                # If the suqare is empty fix the move. 
                if self.board[row][column] == "_": 
                    self.board[row][column] = player.symbol
                    break

            # Check whether this move made the player win. 
            if self.check_winner(player) == 1 or self.check_winner(player) == -1:
                print(f"🎉 Player {player.symbol} won! 🎉")
                is_game_ongoing = False
                break 
            
            # Check if there are any moves left. 
            if self.is_board_full():
                print("🛑 The board is full - Match Draw! 🛑 ")
                break

            # Swap the turn of the player
            if player == player1: player = player2
            else: player = player1

        print()
        print(self.print_board())

    

In [95]:
import random 

class Player():
    """
    Class Player: Provides a structure for creating aplayer for the Tic Tac Toe game. 
    
    Attributes
    ----------
        human: Bool
            Points whether the player is a bot (False) or a human (True). 
        symbol: char
            The symbol associated with this player: either 'O' or 'X'
        maximizing: Bool
            Points whether the player is maximizing or minimizing when making decisions (used for minimax algorithm). 

    Methods
    -------
        None
    """

    def __init__(self, human = False, symbol = 'X', maximizing = False):
        self.human = human
        self.symbol = symbol
        self.maximizing = maximizing



In [96]:
# Play a game

game = TicTacToeGame(3)
player1 = Player(maximizing = False, symbol = 'X')
player2 = Player(maximizing = True, symbol = 'O')

game.play_game(player1 = player1, player2 = player2)

Player X's turn
['_', '_', '_']
['_', '_', '_']
['_', '_', '_']

Player O's turn
['X', '_', '_']
['_', '_', '_']
['_', '_', '_']

Player X's turn
['X', '_', '_']
['O', '_', '_']
['_', '_', '_']

Player O's turn
['X', 'X', '_']
['O', '_', '_']
['_', '_', '_']

Player X's turn
['X', 'X', '_']
['O', 'O', '_']
['_', '_', '_']

🎉 Player X won! 🎉

['X', 'X', 'X']
['O', 'O', '_']
['_', '_', '_']


In [97]:
# Test the methods of the TicTacToeGame class above ^

test_game = TicTacToeGame(3)
player1 = Player()
player2 = Player(maximizing = True, symbol = 'O')


# Draw
test_game.board = [['X', 'O', 'X'], 
                    ['O', 'X', 'X'], 
                    ['_', '_', 'O']]
assert(test_game.check_winner(player1) == 0)

# Row win
test_game.board = [['X', 'X', 'X'], ['_', '_', '_'], ['_', '_', '_']]
assert(test_game.check_winner(player1) == -1)


# No winner, draw
test_game.board = [['O', 'X', 'X'], ['_', '_', '_'], ['_', '_', '_']]
assert(test_game.check_winner(player1) == 0)

# Minimizing agent wins column
test_game.board = [['X', '_', '_'], ['X', '_', '_'], ['X', '_', '_']]
assert(test_game.check_winner(player1) == -1)


# Minimizing agent wins diagonal left to right
test_game.board = [['X', '_', '_'], ['_', 'X', '_'], ['_', '_', 'X']]
assert(test_game.check_winner(player1) == -1)


# Minimizing agent wins diagonal right to left 
test_game.board = [['_', '_', 'X'], ['_', 'X', '_'], ['X', '_', '_']]
assert(test_game.check_winner(player1) == -1)


# I also tested the play_game method by playing the game many times but I can' turn these into test cases since I just looked at the printed strings. 
# It looks correct from what I tested. Did not find any bugs. 

### 4x4 tests

In [98]:
# Tests for board of higher dimension 4 x 4
test_game = TicTacToeGame(4)
player1 = Player()
player2 = Player(maximizing = True, symbol = 'O')

# Draw
test_game.board = [['X', 'O', 'X', '_'], 
                    ['O', 'X', 'X', '_'], 
                    ['_', '_', 'O', '_'],
                     ['X', '_', '_','_']]
assert(test_game.check_winner(player1) == 0)

# Row win
test_game.board = [['X', 'X', 'X', 'X'], ['_', '_', '_','_' ], ['_', '_', '_', '_'],  ['X', '_', '_','_']]
assert(test_game.check_winner(player1) == -1)


# No winner, draw
test_game.board = [['O', 'X', 'X', '_'], ['_', '_', '_','O'], ['_', '_', '_', '_'],  ['X', '_', '_','_']]
assert(test_game.check_winner(player1) == 0)

# Minimizing agent wins column
test_game.board = [['X', '_', '_', '_'], ['X', '_', '_', '_'], ['X', '_', '_','_'], ['X', '_', '_','_']]
assert(test_game.check_winner(player1) == -1)


# Minimizing agent wins diagonal left to right
test_game.board = [['X', '_', '_', '_'], ['_', 'X', '_','_'], ['_', '_', 'X', '_'], ['_', '_', '_', 'X']]
assert(test_game.check_winner(player1) == -1)


# Minimizing agent wins diagonal right to left 
test_game.board = [['_','_', '_', 'X'], ['_','_', 'X', '_'], ['_','X', '_', '_'], ['X','_', '_', '_']]
assert(test_game.check_winner(player1) == -1)


## Stage 2: Person vs (Dumb) Bot

Bot here just picks random squares to play 

In [99]:
import random 

class Player():
    """
    Class Player: Provides a structure for creating aplayer for the Tic Tac Toe game. 
    
    Attributes
    ----------
        human: Bool
            Points whether the player is a bot (False) or a human (True). 
        symbol: char
            The symbol associated with this player: either 'O' or 'X'
        maximizing: Bool
            Points whether the player is maximizing or minimizing when making decisions (used for minimax algorithm). 

    Methods
    -------
        make_move: Return row (int) and column (int) for the chosen move 
    """

    def __init__(self, human = False, symbol = 'X', maximizing = False):
        self.human = human
        self.symbol = symbol
        self.maximizing = maximizing

    ### CHANGE
    def make_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column at random for their move. """

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        random_move = random.choice(possible_moves)

        return random_move[0], random_move[1]

    def _get_possible_moves(self, board_dimension, current_board):
        """ Privated method to get the possible moves where the board squares are empty. """

        moves = []
        # Traverse board to get empty squares
        for i in range(board_dimension):
            for j in range(board_dimension):
                if current_board[i][j] == '_':
                    moves.append((i,j))
        return moves

class TicTacToeGame:
    """
    Class TicTacToeGame: Provides a structure for playing a Tic-Tac-Toe Game
    
    Attributes
    ----------
        board_dimension: int
            The dimension of the board
        board: list of lists
            Used to represent the board. The options are '_' for empty square, 'X', or 'O'
            
    Methods
    -------
        Dunder method str is overwritten.

        print_board: string
            Prints the board in a nicely formatted way
        create_board: list of lists
            Used to initialize the empty board at the beginning of the game
        is_board_full: Bool
            Checks whetehr there are any empty squares left
        check_winner: int
            Checks the board to see if the confuguration is winning. Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player. 
        play_game:
            Starts a tic tac toe game. 
    """

    # Class constructor
    def __init__(self, board_dimension):
        self.board_demension = board_dimension
        self.board = self.create_board()

    def create_board(self):
        """ Method to initialize the empty board in the beginning of the game."""
        return [['_' for i in range(self.board_demension)] for j in range(self.board_demension)]

    # Convert to string. 
    def __str__(self):
        output = ('\n'.join(map(str, self.board)))
        return output

    def print_board(self):
        """Print the board configuration. Used inside other methods (since I can't call __str__ directly"""
        output = ('\n'.join(map(str, self.board)))
        return output

    def is_board_full(self):
        """A function to check if there are any possible moves left (whether the board is filled out or there are any empty squares left)"""
        for i in range(self.board_demension):
            for j in range(self.board_demension):
                if self.board[i][j] == '_':
                    return False
        return True

    def check_winner(self, player):
        """
        Checks whether the current configuration of the board is winning. Acts as a simple evaluation function: 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player.

        Parameters
        __________
        player: Player class instance
        
        Return
        ______
        int: 
            Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player. 
        """
        
        # Check for winning row. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # print("row", i, "column", j) ### DELETE PRINTS USED FOR DEBUGGING
                # Update truth value: if the squares match and they are not empty. 
                condition = condition and (self.board[i][j] == self.board[i][j-1]) and (self.board[i][j] != '_')
                # print("Comparing", self.board[i][j], "and", self.board[i][j-1])
                # print("New condition value is:", condition)
                # If row values don't match, stop checking this row
                if  not condition: 
                    break
            
            # print("CONDITION RIGHT BEFORE IF STATEMENT", condition)
            # If all entries in the row are matching return the value. 
            if condition:
                # print("CONDITION IN IF STATEMENT", condition)
                if player.maximizing: return 1
                else: return -1

        # Check for winning column. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # Update truth value. 
                condition = condition and (self.board[j][i] == self.board[j-1][i])  and (self.board[j][i] != '_')
                # If row values don't match, stop checking this row
                if  not condition: break
            
            # If all entries in the row are matching return the value. 
            if condition:
                if player.maximizing: return 1
                else: return -1

        # Check for winning diagonal (left to right). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][i] == self.board[i-1][i-1]) and self.board[i][i] != "_"
                if not diagonal_condition: break
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            if player.maximizing: return 1
            else: return -1

         # Check for winning diagonal (right to left). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][0-(i+1)] == self.board[i-1][0-i]) and self.board[i-1][0-i] != "_"

            if not diagonal_condition: break
            
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            if player.maximizing: return 1
            else: return -1

        # No winner, it's a draw.
        return 0 

    ### CHANGE : consider whether the player is a bot
    def play_game(self, player1, player2):
        """
        Starts a tic- tac-toe game
        
        Parameters
        _________
        player1: Player class instance
        player2: Player class instance

        Return
        ______
        None
        """

        player = player1

        while True:
            print(f"Player {player.symbol}'s turn")

            print(self.print_board())

            # Approach taken from https://geekflare.com/tic-tac-toe-python-code/ but adapted in my own way. 
            # Get player's turn
            while True:
                # If the player is human ask for input. Otherwise, call bot function to make move. 
                if player.human:
                    row, column = list(
                        map(int, input("Enter row and column numbers to place your symbol (ON AN EMPTY SQUARE): ").split()))
                else:
                    row, column = player.make_move(self.board_demension, self.board)
                print()
                # If the suqare is empty fix the move. 
                if self.board[row][column] == "_": 
                    self.board[row][column] = player.symbol
                    break

            # Check whether this move made the player win. 
            if self.check_winner(player) == 1 or self.check_winner(player) == -1:
                print(f"🎉 Player {player.symbol} won! 🎉")
                is_game_ongoing = False
                break 
            
            # Check if there are any moves left. 
            if self.is_board_full():
                print("🛑 The board is full - Match Draw! 🛑 ")
                break

            # Swap the turn of the player
            if player == player1: player = player2
            else: player = player1

        print()
        print(self.print_board())


In [100]:
# Play a game with player vs bot

game = TicTacToeGame(3)
player1 = Player(human = False, maximizing = False, symbol = 'X')
player2 = Player(human = True, maximizing = True, symbol = 'O')

game.play_game(player1 = player1, player2 = player2)

Player X's turn
['_', '_', '_']
['_', '_', '_']
['_', '_', '_']

Player O's turn
['_', '_', '_']
['_', '_', '_']
['_', '_', 'X']

Player X's turn
['_', '_', '_']
['_', 'O', '_']
['_', '_', 'X']

Player O's turn
['_', '_', 'X']
['_', 'O', '_']
['_', '_', 'X']

Player X's turn
['_', '_', 'X']
['_', 'O', 'O']
['_', '_', 'X']

Player O's turn
['X', '_', 'X']
['_', 'O', 'O']
['_', '_', 'X']

🎉 Player O won! 🎉

['X', '_', 'X']
['O', 'O', 'O']
['_', '_', 'X']


In [101]:
# Play a game bot vs bot

game = TicTacToeGame(3)
player1 = Player(human = False, maximizing = False, symbol = 'X')
player2 = Player(human = False, maximizing = True, symbol = 'O')

game.play_game(player1 = player1, player2 = player2)

Player X's turn
['_', '_', '_']
['_', '_', '_']
['_', '_', '_']

Player O's turn
['_', '_', 'X']
['_', '_', '_']
['_', '_', '_']

Player X's turn
['O', '_', 'X']
['_', '_', '_']
['_', '_', '_']

Player O's turn
['O', '_', 'X']
['_', '_', 'X']
['_', '_', '_']

Player X's turn
['O', '_', 'X']
['_', '_', 'X']
['_', '_', 'O']

Player O's turn
['O', 'X', 'X']
['_', '_', 'X']
['_', '_', 'O']

Player X's turn
['O', 'X', 'X']
['O', '_', 'X']
['_', '_', 'O']

Player O's turn
['O', 'X', 'X']
['O', '_', 'X']
['X', '_', 'O']

🎉 Player O won! 🎉

['O', 'X', 'X']
['O', 'O', 'X']
['X', '_', 'O']


## Stage 3: Player vs Smart Bot

Bot now uses minimax algorithm when making a decision

I divided the `TicTacToeGame` class into a `Board` and a `Game` classes.

In [102]:
class Board:
    """
    Class Board: Provides a structure for the board playing a Tic-Tac-Toe Game. Each board configuarition represents a node in the game tree.
    
    Attributes
    ----------
        board_dimension: int
            The dimension of the board
        board: list of lists
            Used to represent the board. The options are '_' for empty square, 'X', or 'O'
            
    Methods
    -------
        Dunder method str is overwritten.

        print_board: string
            Prints the board in a nicely formatted way
        create_board: list of lists
            Used to initialize the empty board at the beginning of the game
        is_board_full: Bool
            Checks whetehr there are any empty squares left
        check_winner: int
            Checks the board to see if the confuguration is winning. Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player. 
        play_game:
            Starts a tic tac toe game. 
    """

    # Class constructor
    def __init__(self, board_dimension):
        self.board_demension = board_dimension
        self.board = self.create_board()

    def create_board(self):
        """ Method to initialize the empty board in the beginning of the game."""
        return [['_' for i in range(self.board_demension)] for j in range(self.board_demension)]

    # Convert to string. 
    def __str__(self):
        output = ('\n'.join(map(str, self.board)))
        return output

    def print_board(self):
        """Print the board configuration. Used inside other methods (since I can't call __str__ directly"""
        output = ('\n'.join(map(str, self.board)))
        return output

    def is_board_full(self):
        """A function to check if there are any possible moves left (whether the board is filled out or there are any empty squares left)"""
        for i in range(self.board_demension):
            for j in range(self.board_demension):
                if self.board[i][j] == '_':
                    return False
        return True

    def check_winner(self, is_maximizing_player):
        """
        Checks whether the current configuration of the board is winning. Acts as a simple evaluation function: 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player.

        Parameters
        __________
        is_maximizing_player: Bool of the current player
        
        Return
        ______
        int: 
            Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player. 
        """
        
        # Check for winning row. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # print("row", i, "column", j) ### DELETE PRINTS USED FOR DEBUGGING
                # Update truth value: if the squares match and they are not empty. 
                condition = condition and (self.board[i][j] == self.board[i][j-1]) and (self.board[i][j] != '_')
                # print("Comparing", self.board[i][j], "and", self.board[i][j-1])
                # print("New condition value is:", condition)
                # If row values don't match, stop checking this row
                if  not condition: 
                    break
            
            # print("CONDITION RIGHT BEFORE IF STATEMENT", condition)
            # If all entries in the row are matching return the value. 
            if condition:
                # Since we associate X with maximizing player and they won, return 1.
                if self.board[i][j] == 'X': return 1
                # Since we associate O with minimizing player and they won, return -1.
                elif self.board[i][j] == 'O': return -1

        # Check for winning column. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # Update truth value. 
                condition = condition and (self.board[j][i] == self.board[j-1][i])  and (self.board[j][i] != '_')
                # If row values don't match, stop checking this row
                if  not condition: break
            
            # If all entries in the row are matching return the value. 
            if condition:
                # Since we associate X with maximizing player and they won, return 1.
                if self.board[j][i] == 'X': return 1
                # Since we associate O with minimizing player and they won, return -1.
                elif self.board[j][i] == 'O': return -1

        # Check for winning diagonal (left to right). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][i] == self.board[i-1][i-1]) and self.board[i][i] != "_"
                if not diagonal_condition: break
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            # Since we associate X with maximizing player and they won, return 1.
            if self.board[0][0] == 'X': return 1
            # Since we associate O with minimizing player and they won, return -1.
            elif self.board[0][0] == 'O': return -1

         # Check for winning diagonal (right to left). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][0-(i+1)] == self.board[i-1][0-i]) and self.board[i-1][0-i] != "_"

            if not diagonal_condition: break
            
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            # Since we associate X with maximizing player and they won, return 1.
            if self.board[0][-1] == 'X': return 1
            # Since we associate O with minimizing player and they won, return -1.
            elif self.board[0][-1] == 'O': return -1

        # No winner, it's a draw.
        return 0 


class TicTacToeGame:
    """
    Class TicTacToeGame: Provides a structure for a Tic-Tac-Toe Game.
    
    Attributes
    ----------
        None
            
    Methods
    -------
        play_game:
            ParametersL player1 (Player instance), player2 (Player instance), board (Board instance)
            Starts a tic tac toe game. 
    """

    # Class constructor
    def __init__(self):
        pass

    def play_game(self, board, player1, player2):
        """
        Starts a tic- tac-toe game
        
        Parameters
        _________
        player1: Player class instance
        player2: Player class instance
        board: Board class instance

        Return
        ______
        None
        """

        player = player1

        while True:
            print(f"Player {player.symbol}'s turn")

            print(board.print_board())

            # Approach taken from https://geekflare.com/tic-tac-toe-python-code/ but adapted in my own way. 
            # Get player's turn
            while True:
                # If the player is human ask for input. Otherwise, call bot function to make move. 
                if player.human:
                    row, column = list(
                        map(int, input("Enter row and column numbers to place your symbol (ON AN EMPTY SQUARE): ").split()))
                else:
                    row, column = player.make_move(board.board_demension, board.board)
                print()
                # If the suqare is empty fix the move. 
                if board.board[row][column] == "_": 
                    board.board[row][column] = player.symbol
                    break

            # Check whether this move made the player win. 
            if board.check_winner(player) == 1 or board.check_winner(player) == -1:
                print(f"🎉 Player {player.symbol} won! 🎉")
                is_game_ongoing = False
                break 
            
            # Check if there are any moves left. 
            if board.is_board_full():
                print("🛑 The board is full - Match Draw! 🛑 ")
                break

            # Swap the turn of the player
            if player == player1: player = player2
            else: player = player1

        print()
        print(board.print_board())

In [103]:
# Run a game of two bots to check if it is working as expected (like before)

board = Board(3)
game = TicTacToeGame()
player1 = Player(human = False, maximizing = False, symbol = 'X')
player2 = Player(human = False, maximizing = True, symbol = 'O')

game.play_game(player1 = player1, player2 = player2, board = board)

Player X's turn
['_', '_', '_']
['_', '_', '_']
['_', '_', '_']

Player O's turn
['_', 'X', '_']
['_', '_', '_']
['_', '_', '_']

Player X's turn
['_', 'X', '_']
['_', '_', 'O']
['_', '_', '_']

Player O's turn
['X', 'X', '_']
['_', '_', 'O']
['_', '_', '_']

Player X's turn
['X', 'X', '_']
['_', '_', 'O']
['_', 'O', '_']

Player O's turn
['X', 'X', '_']
['_', 'X', 'O']
['_', 'O', '_']

Player X's turn
['X', 'X', 'O']
['_', 'X', 'O']
['_', 'O', '_']

Player O's turn
['X', 'X', 'O']
['X', 'X', 'O']
['_', 'O', '_']

Player X's turn
['X', 'X', 'O']
['X', 'X', 'O']
['O', 'O', '_']

🎉 Player X won! 🎉

['X', 'X', 'O']
['X', 'X', 'O']
['O', 'O', 'X']


Now let's create a method for the minimax algorithm and test it. I have marked all places with changes with a `### CHANGE` comment for easier readability.

In [104]:
import random 
import copy

class Player():
    """
    Class Player: Provides a structure for creating aplayer for the Tic Tac Toe game. 
    
    Attributes
    ----------
        human: Bool
            Points whether the player is a bot (False) or a human (True). 
        symbol: char
            The symbol associated with this player: either 'O' or 'X'
        maximizing: Bool
            Points whether the player is maximizing or minimizing when making decisions (used for minimax algorithm). 

    Methods
    -------
        make_move: Return row (int) and column (int) for the chosen move 
    """

    def __init__(self, human = False, symbol = 'X', maximizing = False):
        self.human = human
        self.symbol = 'X' if maximizing else 'O'
        self.maximizing = maximizing

    ### CHANGE
    def make_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column at random for their move. """

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        random_move = random.choice(possible_moves)

        return random_move[0], random_move[1]

    def _get_possible_moves(self, board_dimension, current_board):
        """ Privated method to get the possible moves where the board squares are empty. """

        moves = []
        # Traverse board to get empty squares
        for i in range(board_dimension):
            for j in range(board_dimension):
                if current_board[i][j] == '_':
                    moves.append((i,j))
        return moves

    ### CHANGE
    def make_random_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column at random for their move. """

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        random_move = random.choice(possible_moves)

        return random_move[0], random_move[1]

    ### CHANGE
    def minimax(self, board, depth, maximizing_player):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s
        """

        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.check_winner(True) != 0 or board.check_winner(False) != 0 or board.is_board_full() == True:
            # print("Condition is met")
            return  board.check_winner(self.maximizing)
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)
            # print("MAXIMIZING PLAYER")
            # print(len(possible_moves))

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                # Make the child node
                # print(move)
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'
                # print(board_copy.board)

                # Evaluate the child node
                eval = self.minimax(board_copy, depth - 1, False)
                # print("🚀 Evaluation", eval, "MaxEval", maxEval)
                maxEval = max(maxEval, eval)

            return maxEval

        else:
            # print("MINIMIZING PLAYER")
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                # print(move)
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'
                # print(board_copy.board)

                # Evaluate the child node
                eval = self.minimax(board_copy, depth - 1, True)
                # print("🚀 Evaluation", eval, "MinEval", minEval)
                minEval = min(minEval, eval)

            return minEval

### Test cases for minimax algorithm.

In [105]:
# Test case for when one move leads to loss -1 and another to a draw 0 so player should choose draw
player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(3)

board.board = [['_', 'O', 'O'],
                ['_', 'O', 'X'],
                ['X', 'X', 'O']]

# Default case (current board - root node)
assert(player1.minimax(board = board, depth=0, maximizing_player=True)==0)
# At depth 2 it should know it is 0 and not -1
assert(player1.minimax(board = board, depth=3, maximizing_player=True)==0)
# At depth 3 it should return the same
assert(player1.minimax(board = board, depth=3, maximizing_player=True)==0)



In [106]:
# Test case when minimizing player should know their most optimal move is to win (-1) rather than draw (0)
board = Board(3)
board.board = [['O', 'X', 'O'],
               ['O', 'O', 'X'],
              ['_', '_', 'X']]
player2 = Player(human=True, symbol='O', maximizing=False)
# At root (cuurent configuration) no one has one and the board is not full yet.
assert(player2.minimax(board = board, depth=0, maximizing_player=False) == 0)
# Minimizing player can win in a single move so depth = 1
assert(player2.minimax(board = board, depth=1, maximizing_player=False) == -1)
# Same check but for a deeper depth
assert(player2.minimax(board = board, depth=8, maximizing_player=False) == -1)

In [107]:
# Test minimax evaluation when for both players the best case scenario is to win (depending on whose move it is)

player1 = Player(human=True, symbol='X', maximizing=True)
player2 = Player(human=True, symbol='O', maximizing=False)
board = Board(3)
board.board = [['O', '_', 'X'],
               ['O', '_', 'X'],
              ['_', '_', '_']]

assert(player1.minimax(board = board, depth=0, maximizing_player=True) == 0)
assert(player1.minimax(board = board, depth=1, maximizing_player=True) == 1)
assert(player2.minimax(board = board, depth=1, maximizing_player=False) == -1)

# Test where best option for X (max) is a draw but for O (min) it's a win in 1 move (depending on whose move it is)
board.board = [['X', 'O', 'X'],
               ['_', 'X', '_'],
              ['O', '_', 'O']]

assert(player1.minimax(board = board, depth=0, maximizing_player=True)== 0)
assert(player1.minimax(board = board, depth=1, maximizing_player=True)== 0)
assert(player1.minimax(board = board, depth=2, maximizing_player=True)== 0)

player2 = Player(human=True, symbol='O', maximizing=False)
assert(player2.minimax(board = board, depth=0, maximizing_player=False) == 0)
assert(player2.minimax(board = board, depth=1, maximizing_player=False) == -1)



In [108]:
# If both players play optimally, then the game will always result in a draw. 
# I'm passing a blank board and depth of 9 (max number of moves) and expect a 0. 
player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(3)
board.board = [['_', '_', '_'],
               ['_', '_', '_'],
              ['_', '_', '_']]

assert(player1.minimax(board = board, depth=9, maximizing_player=True) == 0) # takes about 20 seconds to run

#### Tests for 4x4 board (bigger dimension)

In [109]:
# Test minimax evaluation when for both players the best case scenario is to win (depending on whose move it is)

player1 = Player(human=True, symbol='X', maximizing=True)
player2 = Player(human=True, symbol='O', maximizing=False)
board = Board(4)
board.board = [['O', 'O', 'O', '_'],
               ['X', 'X', 'X', '_'],
               ['_', '_', '_', '_'],
               ['_', '_', '_', '_']]

assert(player1.minimax(board = board, depth=0, maximizing_player=True) == 0)
assert(player1.minimax(board = board, depth=1, maximizing_player=True) == 1)
assert(player2.minimax(board = board, depth=1, maximizing_player=False) == -1)

In [29]:
# Test where best option for X (max) is a draw but for O (min) it's a win in 1 move (depending on whose move it is)
board = Board(4)
board.board = [['O', 'O', 'O', '_'],
               ['X', 'O', 'X', '_'],
               ['O', 'X', 'X', 'O'],
               ['O', 'X', 'O', 'X']]

player1 = Player(human=True, symbol='X', maximizing=True)
player2 = Player(human=True, symbol='O', maximizing=False)

assert(player1.minimax(board = board, depth=0, maximizing_player=True)== 0)
assert(player1.minimax(board = board, depth=1, maximizing_player=True)== 0)
assert(player1.minimax(board = board, depth=2, maximizing_player=True)== 0)

player2 = Player(human=True, symbol='O', maximizing=False)
assert(player2.minimax(board = board, depth=0, maximizing_player=False) == 0)
assert(player2.minimax(board = board, depth=1, maximizing_player=False) == -1)

In [ ]:
### !!! I could not verify this test because the cell takes too long (15+ minutes to run)
# If both players play optimally, then the game will always result in a draw. I am not sure if this still holds for a board of higher dimension.
# I'm passing a blank board and depth of 9 (max number of moves) and expect a 0. 
player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(4)
board.board = [['_', '_', '_', '_'],
               ['_', '_', '_', '_'],
               ['_', '_', '_', '_'],
               ['_', '_', '_', '_']]

assert(player1.minimax(board = board, depth=16, maximizing_player=True) == 0)

### Making a move based on minimax 
Now let's change the `make_move` method so that the bot can make the best move by using the minimax algorithm. 

In [110]:
import random 
import copy

class Player():
    """
    Class Player: Provides a structure for creating aplayer for the Tic Tac Toe game. 
    
    Attributes
    ----------
        human: Bool
            Points whether the player is a bot (False) or a human (True). 
        symbol: char
            The symbol associated with this player: either 'O' or 'X'
        maximizing: Bool
            Points whether the player is maximizing or minimizing when making decisions (used for minimax algorithm). 

    Methods
    -------
        make_move: Return row (int) and column (int) for the chosen move 
        _get_possible_moves: Return all possible moves (2-tuples of ints) given current board configuration
        make_random_move: Return row (int) and column (int) for randomly chosen move 
        minimax: Return evaluation of current board configuration (int) using minimax algorithm
    """

    def __init__(self, human = False, symbol = 'X', maximizing = False):
        self.human = human
        self.symbol = 'X' if maximizing else 'O'
        self.maximizing = maximizing

    ### CHANGE
    def make_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column for their move using the minimax algorithm. """

        if self.maximizing: best_move_value = -9999
        else: best_move_value = 9999

        best_move = None

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        # Traverse all possible moces.
        for current_move in possible_moves:
            # Make the child node
            board_copy = copy.deepcopy(board)
            board_copy.board[current_move[0]][current_move[1]] = self.symbol

            # Evaluate the child node - the evaluation depends on the depth
            # The bigger the depth is the better the decision. 
            current_move_value = self.minimax(board = board_copy, depth=3, maximizing_player= not self.maximizing)

            # If it is a better move save it.
            if self.maximizing and current_move_value > best_move_value:
                best_move = current_move
                best_move_value = current_move_value
            elif not self.maximizing and current_move_value < best_move_value:
                best_move = current_move
                best_move_value = current_move_value

        return best_move[0], best_move[1]

    def _get_possible_moves(self, board_dimension, current_board):
        """ Privated method to get the possible moves where the board squares are empty. """

        moves = []
        # Traverse board to get empty squares
        for i in range(board_dimension):
            for j in range(board_dimension):
                if current_board[i][j] == '_':
                    moves.append((i,j))
        return moves

    def make_random_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column at random for their move. """

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        random_move = random.choice(possible_moves)

        return random_move[0], random_move[1]

    def minimax(self, board, depth, maximizing_player):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s

        Parameters
        ----------
        board: Board instance
            The current board configuration, child node in game tree
        depth: int
            The depth level in the game tree
        maximizing_player: Bool
            Indicates whether it is MAX or MIN's turn

        Returns
        -------
        value, steps: a tuple with evaluation of board configuration (child node) and number of child nodes explored
        """

        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.check_winner(True) != 0 or board.check_winner(False) != 0 or board.is_board_full() == True:
            return  board.check_winner(self.maximizing)
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'

                # Evaluate the child node
                eval = self.minimax(board_copy, depth - 1, False)
                maxEval = max(maxEval, eval)

            return maxEval

        else:
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'

                # Evaluate the child node
                eval = self.minimax(board_copy, depth - 1, True)
                minEval = min(minEval, eval)

            return minEval

Let's check if the code above is working 

In [111]:
# Run a game of two bots to check if it is working as expected (like before)
# We expect it to end in a draw since both players play optimally, which is what happens.

board = Board(3)
game = TicTacToeGame()
player1 = Player(human = False, maximizing = True, symbol = 'X')
player2 = Player(human = False, maximizing = False, symbol = 'O')

game.play_game(player1 = player1, player2 = player2, board = board)

Player X's turn
['_', '_', '_']
['_', '_', '_']
['_', '_', '_']

Player O's turn
['X', '_', '_']
['_', '_', '_']
['_', '_', '_']

Player X's turn
['X', 'O', '_']
['_', '_', '_']
['_', '_', '_']

Player O's turn
['X', 'O', 'X']
['_', '_', '_']
['_', '_', '_']

Player X's turn
['X', 'O', 'X']
['_', 'O', '_']
['_', '_', '_']

Player O's turn
['X', 'O', 'X']
['_', 'O', '_']
['_', 'X', '_']

Player X's turn
['X', 'O', 'X']
['O', 'O', '_']
['_', 'X', '_']

Player O's turn
['X', 'O', 'X']
['O', 'O', 'X']
['_', 'X', '_']

Player X's turn
['X', 'O', 'X']
['O', 'O', 'X']
['_', 'X', 'O']

🛑 The board is full - Match Draw! 🛑 

['X', 'O', 'X']
['O', 'O', 'X']
['X', 'X', 'O']


In [112]:
# Same test but with a board of higher dimension 4x4 

board = Board(4)
game = TicTacToeGame()
player1 = Player(human = False, maximizing = True, symbol = 'X')
player2 = Player(human = False, maximizing = False, symbol = 'O')

game.play_game(player1 = player1, player2 = player2, board = board)

Player X's turn
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player O's turn
['X', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player X's turn
['X', 'O', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player O's turn
['X', 'O', 'X', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player X's turn
['X', 'O', 'X', 'O']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player O's turn
['X', 'O', 'X', 'O']
['X', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player X's turn
['X', 'O', 'X', 'O']
['X', 'O', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player O's turn
['X', 'O', 'X', 'O']
['X', 'O', 'X', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player X's turn
['X', 'O', 'X', 'O']
['X', 'O', 'X', 'O']
['_', '_', '_', '_']
['_', '_', '_', '_']

Player O's turn
['X', 'O', 'X', 'O']
['X', 'O', 'X', 'O']
['X', '_', '_', '_']
['_', '_', '

In [63]:
# Same test but with a board of higher dimension 6x6 - takes very long to run, so I interrupted the kernel

board = Board(6)
game = TicTacToeGame()
player1 = Player(human = False, maximizing = True, symbol = 'X')
player2 = Player(human = False, maximizing = False, symbol = 'O')

game.play_game(player1 = player1, player2 = player2, board = board)

Player X's turn
['_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_']
['_', '_', '_', '_', '_', '_']
36
Current move (0, 0)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (0, 1)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (0, 2)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (0, 3)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (0, 4)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (0, 5)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (1, 0)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (1, 1)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (1, 2)
Current move evaluation 0
Best move is (0, 0) with evaluation 0
Current move (1, 3)
Current move evalua

KeyboardInterrupt: 

## Stage 4: Alpha beta pruning

Let's update the `Player` class and add a method that implements the minimax algorithm but with alpha beta pruning `optimized_minimax()`. I have also added a `step` parameter to `minimax()` and `optimized_minimax()` so I can compare how many children nodes are explored. We expect that the alpha beta pruning makes the search faster as it has to go through less children. 

In [113]:
import random 
import copy

class Player():
    """
    Class Player: Provides a structure for creating aplayer for the Tic Tac Toe game. 
    
    Attributes
    ----------
        human: Bool
            Points whether the player is a bot (False) or a human (True). 
        symbol: char
            The symbol associated with this player: either 'O' or 'X'
        maximizing: Bool
            Points whether the player is maximizing or minimizing when making decisions (used for minimax algorithm). 

    Methods
    -------
        make_move: Return row (int) and column (int) for the chosen move 
        _get_possible_moves: Return all possible moves (2-tuples of ints) given current board configuration
        make_random_move: Return row (int) and column (int) for randomly chosen move 
        minimax: Return evaluation of current board configuration (int) using minimax algorithm
        optimized_minimax: Return evaluation of current board configuration (int) using minimax algorithm with alpha beta pruning
    """

    def __init__(self, human = False, symbol = 'X', maximizing = False):
        self.human = human
        self.symbol = 'X' if maximizing else 'O'
        self.maximizing = maximizing

    def make_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column for their move using the minimax algorithm. """

        if self.maximizing: best_move_value = -9999
        else: best_move_value = 9999

        best_move = None

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        # Traverse all possible moces.
        for current_move in possible_moves:
            # Make the child node
            board_copy = copy.deepcopy(board)
            board_copy.board[current_move[0]][current_move[1]] = self.symbol

            # Evaluate the child node - the evaluation depends on the depth
            # The bigger the depth is the better the decision. 
            current_move_value = self.minimax(board = board_copy, depth=3, maximizing_player= not self.maximizing)

            # If it is a better move save it.
            if self.maximizing and current_move_value > best_move_value:
                best_move = current_move
                best_move_value = current_move_value
            elif not self.maximizing and current_move_value < best_move_value:
                best_move = current_move
                best_move_value = current_move_value

        return best_move[0], best_move[1]

    def _get_possible_moves(self, board_dimension, current_board):
        """ Privated method to get the possible moves where the board squares are empty. """

        moves = []
        # Traverse board to get empty squares
        for i in range(board_dimension):
            for j in range(board_dimension):
                if current_board[i][j] == '_':
                    moves.append((i,j))
        return moves

    def make_random_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column at random for their move. """

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        random_move = random.choice(possible_moves)

        return random_move[0], random_move[1]

    ### CHANGED - added steps to indicate child nodes exlored
    def minimax(self, board, depth, maximizing_player):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s

        Parameters
        ----------
        board: Board instance
            The current board configuration, child node in game tree
        depth: int
            The depth level in the game tree
        maximizing_player: Bool
            Indicates whether it is MAX or MIN's turn

        Returns
        -------
        value, steps: a tuple with evaluation of board configuration (child node) and number of child nodes explored
        """
        steps = 0
        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.check_winner(True) != 0 or board.check_winner(False) != 0 or board.is_board_full() == True:
            # I decided not to count this as a step and only the new children nodes that are explored.
            return  board.check_winner(self.maximizing), steps
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                
                steps += 1 # Each valid child node (board configuration) explored is another step.
                
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'

                # Evaluate the child node
                result = self.minimax(board_copy, depth - 1, False)
                eval = result[0]
                steps += result[1]
                maxEval = max(maxEval, eval)

            return maxEval, steps

        else:
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999
            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:

                steps += 1 # Each valid child node (board configuration) explored is another step.
                
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'

                # Evaluate the child node
                result = self.minimax(board_copy, depth - 1, True)
                eval = result[0]
                steps += result[1]
                minEval = min(minEval, eval)

            return minEval, steps

    ### CHANGE
    def optimized_minimax(self, board, depth, alpha, beta, maximizing_player):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm with alpha beta pruning.
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s

        Parameters
        ----------
        board: Board instance
            The current board configuration, child node in game tree
        depth: int
            The depth level in the game tree
        alpha: int
            Parameter with a value associated with the max nodes
        beta: int 
            Parameter with a value associated with the min nodes
        maximizing_player: Bool
            Indicates whether it is MAX or MIN's turn

        Returns
        -------
        value, steps: a tuple with evaluation of board configuration (child node) and number of child nodes explored
        """
        steps = 0
        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.check_winner(True) != 0 or board.check_winner(False) != 0 or board.is_board_full() == True:
            return  board.check_winner(self.maximizing), steps
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:

                steps += 1 # Each valid child node (board configuration) explored is another step.

                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'

                # Evaluate the child node
                result = self.optimized_minimax(board_copy, depth - 1, alpha, beta, False)
                eval = result[0]
                steps += result[1]

                maxEval = max(maxEval, eval)

                ### CHANGE
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
                ###

            return maxEval, steps

        else:
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                
                steps += 1 # Each valid child node (board configuration) explored is another step.

                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'

                # Evaluate the child node
                result = self.optimized_minimax(board_copy, depth - 1, alpha, beta, True)
                eval = result[0]
                steps += result[1]

                minEval = min(minEval, eval)

                ### CHANGE
                beta = min(beta, eval)
                if beta <= alpha:
                    break
                ###

                

            return minEval, steps

### Test cases for the step counter and alpha beta pruning

In [114]:
# Test case for when one move leads to loss -1 and another to a draw 0 so player should choose draw
# We can easily see that there are 4 nodes that need to be explored - 2 possible moves for X and 2 possible moves for O player = 4 in total
player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(3)

board.board = [['_', 'O', 'O'],
                ['_', 'O', 'X'],
                ['X', 'X', 'O']]

# Default case (current board - root node)
assert(player1.minimax(board = board, depth=0, maximizing_player=True) == (0,0)) # 0 for a draw and 0 children nodes explored (steps)
# At depth 2 it should know it is 0 and not -1
assert(player1.minimax(board = board, depth=2, maximizing_player=True) == (0,4)) # 0 for a draw and 4 children nodes explored (steps)
# At depth 3 it should return the same
assert(player1.minimax(board = board, depth=3, maximizing_player=True) == (0,4)) # should be the same as previous depth since only 2 moves are possible.

In [115]:
# Test case for when any move leads to a loss
player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(3)

board.board = [['_', 'O', 'O'],
                ['_', 'O', 'X'],
                ['_', 'X', 'O']]

# Default case (current board - root node)
assert(player1.minimax(board = board, depth=3, maximizing_player=True) == (-1, 11)) # -1 for loss and 11 children nodes explored (steps)

The traversal tree of the configuration above is the following. The coordinates represent the move made by the player. Each move leads to a new baord configuration which is a new child node explored. If we count the nodes (except for the root) we can see that 11 children should be explored.
```
root
|
|_ 0,0 
|    |_  1,0 
|    |    |_ 2,0 (draw) 
|    |
|    |_ 2,0 (Player O wins)
|
|_ 1,0
|    |_ 0,0 (Player O wins)
|    |_ 2,0 (Player O wins)
|
|_ 2,0
    |_0,0 (Player O wins)
    |_ 1,0 (Player O wins)
        |_ 0,0 (draw)

```


In [116]:
# Test case for when the maximizing player can draw or lose - this is the board configuration from the PCW for session 6 (3.2)
# The tree we drew then explored 13 children nodes - and this is what the function returns for steps so it seems correct
player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(3)

board.board = [['O', 'O', 'X'],
                ['_', 'X', '_'],
                ['O', 'X', '_']]

# Default case (current board - root node)
assert(player1.minimax(board = board, depth=3, maximizing_player=True) == (0,13))

The game tree for the example above is the following
![Image](https://github.com/vickipetrova/tic-tac-toe/blob/main/IMG_0852.jpg?raw=true)

Now let's see how many nodes will be explored with the alpha beta pruning. We expect 9 nodes (4 less) to be searched. 

This is because the search will go down the left most branch and explore the 5 nodes there. Node B's value will now be a 0. So at the root player MAX expects a value `>= 0`. 

Then the search will go down the second branch to node C. Then in node G it terminates (MIN player won) and the static value is -1. That means that for node C the value is `<=-1` since player MIN makes the decision. That means that we do not need to explore any other options in that treee because the MAX player already has a better option (the left-most branch) with a score of 0 which is bigger than -1. This way we pruned nodes H and M. 

Then, it explored node D. Then, node in I it terminates (MIN player won) and the static value is -1. Same logic as before is valid. It does not need to explore J and N. 

The game tree is the following:
![Image](https://github.com/vickipetrova/tic-tac-toe/blob/main/IMG_0853.jpg?raw=true)

In [117]:
player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(3)

board.board = [['O', 'O', 'X'],
                ['_', 'X', '_'],
                ['O', 'X', '_']]

# Default case (current board - root node)
assert(player1.optimized_minimax(board = board, depth=3, alpha = -9999, beta = 9999, maximizing_player=True) == (0,9))

In [118]:
# Test cases for higher dimension 4x4 board
# This test runs the normal minimax algorithm and then the minimax with alpha beta pruning and makes sure the latter explores less nodes.

player1 = Player(human=True, symbol='X', maximizing=True)
board = Board(4)

boards = [ [['O', 'O', 'X', '_'],
            ['_', 'X', '_', 'O'],
            ['O', 'X', 'X', 'O'],
            ['O', 'X', '_', 'O']], 

            [['X', 'X', 'O', '_'],
            ['_', 'O', '_', 'X'],
            ['X', 'O', 'O', 'X'],
            ['X', 'O', '_', 'X']],

            [['O', 'O', 'O', '_'],
            ['O', 'O', '_', 'X'],
            ['X', 'O', 'O', 'O'],
            ['O', 'O', '_', 'X']]
        ]

for i in range (2):

    board.board = boards[i]

    steps_normal = player1.minimax(board = board, depth=4, maximizing_player=True)
    steps_optimized = player1.optimized_minimax(board = board, depth=4, alpha = -9999, beta = 9999, maximizing_player=True)

    assert steps_optimized <= steps_normal

## Stage 5: Evaluation function

I added `h1` as a new evaluation function. It prioritizes faster winning moves by taking into account the value of the depth from the evaluated score. 

If MAX player has two winning moves one that takes 1 move and another that takes 5 moves. Before it would return the same value +1 for both. 
But now it will return +100 - 1 = 99 for 1 move and +100 - 5 = 95 for the second move. This way it prioritizes winning faster.

On the other hand if MAX player is going to lose it will prioritize making the game as long as possible as it will choose the bigger scores, meaning that MIN played more moves. 

An analogous logic is true for the MIN player but with switched +/- signs.

Idea taken from this resource: https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-3-tic-tac-toe-ai-finding-optimal-move/

It does not speed up the search at first. It chooses the fastest path to win (least number of moves and least number of child nodes). And it chooses the slowest way to lose (biggest number of moves and child nodes).

In [119]:
class Board:
    """
    Class Board: Provides a structure for the board playing a Tic-Tac-Toe Game. Each board configuarition represents a node in the game tree.
    
    Attributes
    ----------
        board_dimension: int
            The dimension of the board
        board: list of lists
            Used to represent the board. The options are '_' for empty square, 'X', or 'O'
            
    Methods
    -------
        Dunder method str is overwritten.

        print_board: string
            Prints the board in a nicely formatted way
        create_board: list of lists
            Used to initialize the empty board at the beginning of the game
        is_board_full: Bool
            Checks whetehr there are any empty squares left
        check_winner: int
            Checks the board to see if the confuguration is winning. Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player.  
        h1: int
            Evaluation function using a heuristic to prioritize faster winning moves or slower losing moves by considering the depth level.
    """

    # Class constructor
    def __init__(self, board_dimension):
        self.board_demension = board_dimension
        self.board = self.create_board()

    def create_board(self):
        """ Method to initialize the empty board in the beginning of the game."""
        return [['_' for i in range(self.board_demension)] for j in range(self.board_demension)]

    # Convert to string. 
    def __str__(self):
        output = ('\n'.join(map(str, self.board)))
        return output

    def print_board(self):
        """Print the board configuration. Used inside other methods (since I can't call __str__ directly"""
        output = ('\n'.join(map(str, self.board)))
        return output

    def is_board_full(self):
        """A function to check if there are any possible moves left (whether the board is filled out or there are any empty squares left)"""
        for i in range(self.board_demension):
            for j in range(self.board_demension):
                if self.board[i][j] == '_':
                    return False
        return True

    def check_winner(self, is_maximizing_player):
        """
        Checks whether the current configuration of the board is winning. Acts as a simple evaluation function: 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player.

        Parameters
        __________
        is_maximizing_player: Bool of the current player
        
        Return
        ______
        int: 
            Return 0 for draw, -1 for win for minimizing player, +1 for win for maximizing player. 
        """
        
        # Check for winning row. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # print("row", i, "column", j) ### DELETE PRINTS USED FOR DEBUGGING
                # Update truth value: if the squares match and they are not empty. 
                condition = condition and (self.board[i][j] == self.board[i][j-1]) and (self.board[i][j] != '_')
                # print("Comparing", self.board[i][j], "and", self.board[i][j-1])
                # print("New condition value is:", condition)
                # If row values don't match, stop checking this row
                if  not condition: 
                    break
            
            # print("CONDITION RIGHT BEFORE IF STATEMENT", condition)
            # If all entries in the row are matching return the value. 
            if condition:
                # Since we associate X with maximizing player and they won, return 1.
                if self.board[i][j] == 'X': return 1
                # Since we associate O with minimizing player and they won, return -1.
                elif self.board[i][j] == 'O': return -1

        # Check for winning column. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # Update truth value. 
                condition = condition and (self.board[j][i] == self.board[j-1][i])  and (self.board[j][i] != '_')
                # If row values don't match, stop checking this row
                if  not condition: break
            
            # If all entries in the row are matching return the value. 
            if condition:
                # Since we associate X with maximizing player and they won, return 1.
                if self.board[j][i] == 'X': return 1
                # Since we associate O with minimizing player and they won, return -1.
                elif self.board[j][i] == 'O': return -1

        # Check for winning diagonal (left to right). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][i] == self.board[i-1][i-1]) and self.board[i][i] != "_"
                if not diagonal_condition: break
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            # Since we associate X with maximizing player and they won, return 1.
            if self.board[0][0] == 'X': return 1
            # Since we associate O with minimizing player and they won, return -1.
            elif self.board[0][0] == 'O': return -1

         # Check for winning diagonal (right to left). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][0-(i+1)] == self.board[i-1][0-i]) and self.board[i-1][0-i] != "_"

            if not diagonal_condition: break
            
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            # Since we associate X with maximizing player and they won, return 1.
            if self.board[0][-1] == 'X': return 1
            # Since we associate O with minimizing player and they won, return -1.
            elif self.board[0][-1] == 'O': return -1

        # No winner, it's a draw.
        return 0 

    ### CHANGE
    def h1(self, is_maximizing_player, depth):
        """
        Evaluation function. It prioritizes faster winning moves by taking into account the value of the depth from the evaluated score. 

        If MAX player has two winning moves one that takes 1 move and another that takes 5 moves. Before it would return the same value +1 for both. 
        But now it will return +100 - 1 = 99 for 1 move and +100 - 5 = 95 for the second move. This way it prioritizes winning faster.

        On the other hand if MAX player is going to lose it will prioritize making the game as long as possible as it will choose the bigger scores, meaning that MIN played more moves. 

        An analogous logic is true for the MIN player but with switched +/- signs.

        Idea taken from this resource: https://www.geeksforgeeks.org/minimax-algorithm-in-game-theory-set-3-tic-tac-toe-ai-finding-optimal-move/

        Parameters
        __________
        is_maximizing_player: Bool of the current player
        depth: int the current depth level of the game tree
        
        Return
        ______
        int: 
            Return 0 for draw, -100 + depth for win for minimizing player, +100 - depth for win for maximizing player. 
        """
        
        # Check for winning row. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # print("row", i, "column", j) ### DELETE PRINTS USED FOR DEBUGGING
                # Update truth value: if the squares match and they are not empty. 
                condition = condition and (self.board[i][j] == self.board[i][j-1]) and (self.board[i][j] != '_')
                # print("Comparing", self.board[i][j], "and", self.board[i][j-1])
                # print("New condition value is:", condition)
                # If row values don't match, stop checking this row
                if  not condition: 
                    break
            
            # print("CONDITION RIGHT BEFORE IF STATEMENT", condition)
            # If all entries in the row are matching return the value. 
            if condition:
                # Since we associate X with maximizing player and they won, return 1.
                if self.board[i][j] == 'X': return 100 - depth # CHANGE
                # Since we associate O with minimizing player and they won, return -1.
                elif self.board[i][j] == 'O': return -100 + depth # CHANGE

        # Check for winning column. 
        condition = None
        for i in range(self.board_demension):
            condition = True
            for j in range(1, self.board_demension):
                # Update truth value. 
                condition = condition and (self.board[j][i] == self.board[j-1][i])  and (self.board[j][i] != '_')
                # If row values don't match, stop checking this row
                if  not condition: break
            
            # If all entries in the row are matching return the value. 
            if condition:
                # Since we associate X with maximizing player and they won, return 1.
                if self.board[j][i] == 'X': return 100 - depth # CHANGE
                # Since we associate O with minimizing player and they won, return -1.
                elif self.board[j][i] == 'O': return -100 + depth # CHANGE

        # Check for winning diagonal (left to right). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][i] == self.board[i-1][i-1]) and self.board[i][i] != "_"
                if not diagonal_condition: break
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            # Since we associate X with maximizing player and they won, return 1.
            if self.board[0][0] == 'X': return 100 - depth # CHANGE
            # Since we associate O with minimizing player and they won, return -1.
            elif self.board[0][0] == 'O': return -100 + depth # CHANGE

         # Check for winning diagonal (right to left). 
        diagonal_condition = True
        for i in range(self.board_demension):
            if i == 0: continue
            else:
                diagonal_condition = diagonal_condition and (self.board[i][0-(i+1)] == self.board[i-1][0-i]) and self.board[i-1][0-i] != "_"

            if not diagonal_condition: break
            
            
        # If all entries in the diagonal are matching return the value. 
        if diagonal_condition:
            # Since we associate X with maximizing player and they won, return 1.
            if self.board[0][-1] == 'X': return 100 - depth # CHANGE
            # Since we associate O with minimizing player and they won, return -1.
            elif self.board[0][-1] == 'O': return -100 + depth # CHANGE

        # No winner, it's a draw.
        return 0 

In [120]:
import random 
import copy

class Player():
    """
    Class Player: Provides a structure for creating aplayer for the Tic Tac Toe game. 
    
    Attributes
    ----------
        human: Bool
            Points whether the player is a bot (False) or a human (True). 
        symbol: char
            The symbol associated with this player: either 'O' or 'X'
        maximizing: Bool
            Points whether the player is maximizing or minimizing when making decisions (used for minimax algorithm). 

    Methods
    -------
        make_move: Return row (int) and column (int) for the chosen move 
        _get_possible_moves: Return all possible moves (2-tuples of ints) given current board configuration
        make_random_move: Return row (int) and column (int) for randomly chosen move 
        minimax: Return evaluation of current board configuration (int) using minimax algorithm
        optimized_minimax: Return evaluation of current board configuration (int) using minimax algorithm with alpha beta pruning
    """

    def __init__(self, human = False, symbol = 'X', maximizing = False):
        self.human = human
        self.symbol = 'X' if maximizing else 'O'
        self.maximizing = maximizing

    ### CHANGED
    def make_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column for their move using the minimax algorithm. """

        if self.maximizing: best_move_value = -9999
        else: best_move_value = 9999

        best_move = None

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        # Traverse all possible moces.
        for current_move in possible_moves:
            # Make the child node
            board_copy = copy.deepcopy(board)
            board_copy.board[current_move[0]][current_move[1]] = self.symbol

            # Evaluate the child node - the evaluation depends on the depth
            # The bigger the depth is the better the decision. 
            current_move_value = self.optimized_minimax(board = board_copy, depth=3, maximizing_player= not self.maximizing, alpha =-9999, beta = 9999, max_depth=3)[0] # Changed, get the evaluation not the steps
            
            # If it is a better move save it.
            if self.maximizing and current_move_value > best_move_value:
                best_move = current_move
                best_move_value = current_move_value
            elif not self.maximizing and current_move_value < best_move_value:
                best_move = current_move
                best_move_value = current_move_value

        return best_move[0], best_move[1]

    def _get_possible_moves(self, board_dimension, current_board):
        """ Privated method to get the possible moves where the board squares are empty. """

        moves = []
        # Traverse board to get empty squares
        for i in range(board_dimension):
            for j in range(board_dimension):
                if current_board[i][j] == '_':
                    moves.append((i,j))
        return moves

    def make_random_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column at random for their move. """

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        random_move = random.choice(possible_moves)

        return random_move[0], random_move[1]

    def minimax(self, board, depth, maximizing_player):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s

        Parameters
        ----------
        board: Board instance
            The current board configuration, child node in game tree
        depth: int
            The depth level in the game tree
        maximizing_player: Bool
            Indicates whether it is MAX or MIN's turn

        Returns
        -------
        value, steps: a tuple with evaluation of board configuration (child node) and number of child nodes explored
        """
        steps = 0

        # CHANGE to use heuristic
        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.h1(True, depth) != 0 or board.h1(False, depth) != 0 or board.is_board_full() == True:
            # I decided not to count this as a step and only the new children nodes that are explored.
            return  board.h1(self.maximizing, depth), steps
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                
                steps += 1 # Each valid child node (board configuration) explored is another step.
                
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'

                # Evaluate the child node
                result = self.minimax(board_copy, depth - 1, False)
                eval = result[0]
                steps += result[1]
                maxEval = max(maxEval, eval)

            return maxEval, steps

        else:
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999
            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:

                steps += 1 # Each valid child node (board configuration) explored is another step.
                
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'

                # Evaluate the child node
                result = self.minimax(board_copy, depth - 1, True)
                eval = result[0]
                steps += result[1]
                minEval = min(minEval, eval)

            return minEval, steps
    ### CHANGED
    def optimized_minimax(self, board, depth, alpha, beta, maximizing_player, max_depth):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm with alpha beta pruning.
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s

        Parameters
        ----------
        board: Board instance
            The current board configuration, child node in game tree
        depth: int
            The depth level in the game tree
        alpha: int
            Parameter with a value associated with the max nodes
        beta: int 
            Parameter with a value associated with the min nodes
        maximizing_player: Bool
            Indicates whether it is MAX or MIN's turn

        Returns
        -------
        value, steps: a tuple with evaluation of board configuration (child node) and number of child nodes explored
        """
        steps = 0
        
        # CHANGE to use heuristic
        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.h1(True, depth) != 0 or board.h1(False, depth) != 0 or board.is_board_full() == True:
            return board.h1(self.maximizing, max_depth-depth), steps
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:

                steps += 1 # Each valid child node (board configuration) explored is another step.

                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'

                # Evaluate the child node
                result = self.optimized_minimax(board_copy, depth - 1, alpha, beta, False, max_depth)
                eval = result[0]
                steps += result[1]

                maxEval = max(maxEval, eval)

                alpha = max(alpha, eval)
                if beta <= alpha:
                    break

            return maxEval, steps

        else:
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                
                steps += 1 # Each valid child node (board configuration) explored is another step.

                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'

                # Evaluate the child node
                result = self.optimized_minimax(board_copy, depth - 1, alpha, beta, True, max_depth)
                eval = result[0]
                steps += result[1]

                minEval = min(minEval, eval)

                beta = min(beta, eval)
                if beta <= alpha:
                    break

                
            return minEval, steps

### Test cases for `h1`

In [121]:
player1 = Player(human=False, symbol='X', maximizing=True)
board = Board(3)

board.board = [['_', 'O', 'X'],
                ['_', 'O', 'X'],
                ['X', 'X', '_']]

# Default case (current board - root node)
assert(player1.optimized_minimax(board = board, depth=3, alpha = -9999, beta = 9999, maximizing_player=True, max_depth=3) == (99, 9))

# The maximizing player should win in a single move (2,2) rather than just playing the first move (0,0) that wins.
game = TicTacToeGame()
player2 = Player(human = True, maximizing = False, symbol = 'O')

game.play_game(player1 = player1, player2 = player2, board = board)

Player X's turn
['_', 'O', 'X']
['_', 'O', 'X']
['X', 'X', '_']

🎉 Player X won! 🎉

['_', 'O', 'X']
['_', 'O', 'X']
['X', 'X', 'X']


Another possible evaluation function is one that I found in this resource https://www3.ntu.edu.sg/home/ehchua/programming/java/javagame_tictactoe_ai.html#:~:text=1.5%20Minimax%20Search%20Algorithm,-Reference%3A%20Wiki%20%22Minimax&text=For%20Tic%2DTac%2DToe%2C%20the%20function%20could%20be%20as,%2Da%2Dline%20for%20computer. 

![heuristic](https://github.com/vickipetrova/tic-tac-toe/blob/main/Screenshot%202022-12-21%20at%2011.49.59%20PM.png?raw=true)

It is a good heuristic as it makes an inference on which moves are more likely to win. However its implementation is more complex and I did not have time to put it into code. Nevertheless it would be a valuable addition to this project.

## Memoization

We can use memoization to make the bot make decisions faster. To do this I will create decoration method for the `Player` class called `_memoize_move`. I reference my code from Assignment 2 when creating the decorator.

In [127]:
import random 
import copy

class Player():
    """
    Class Player: Provides a structure for creating aplayer for the Tic Tac Toe game. 
    
    Attributes
    ----------
        human: Bool
            Points whether the player is a bot (False) or a human (True). 
        symbol: char
            The symbol associated with this player: either 'O' or 'X'
        maximizing: Bool
            Points whether the player is maximizing or minimizing when making decisions (used for minimax algorithm). 

    Methods
    -------
        make_move: Return row (int) and column (int) for the chosen move 
        _get_possible_moves: Return all possible moves (2-tuples of ints) given current board configuration
        make_random_move: Return row (int) and column (int) for randomly chosen move 
        minimax: Return evaluation of current board configuration (int) using minimax algorithm
        optimized_minimax: Return evaluation of current board configuration (int) using minimax algorithm with alpha beta pruning
    """

    def __init__(self, human = False, symbol = 'X', maximizing = False):
        self.human = human
        self.symbol = 'X' if maximizing else 'O'
        self.maximizing = maximizing

    ### CHANGE
    # Decorator. 
    def _memoize_move(some_function):
        """
        This function is used as a decorator for the make_move function. It stores the best_move tuple in a database and
        access it if it has already been computed for the current state (board configuration or node). Otherwise it calls the make_move function
        to find and store it. 
        
        Parameters
        ----------
            some_function: function
                Some evaluation function (e.g. h1, check_winner).
        Returns
        -------
            int
                Returns the helper function that either fetches the value if available in the databse
                or calls the function to calculate the value, stores it in a dictionary, and then returns it. 
        """
        # Store all values in a dictionary. 
        move_db = {}
        print(move_db)

        def helper(self, board_dimension, current_board):
            # If the state hasn't been encountered before, store its value. 
            if str(current_board) not in move_db: 
                move_db[str(current_board)] = some_function(self, board_dimension, current_board)
            return move_db[str(current_board)]
        return helper

    ### CHANGED 
    @ _memoize_move
    def make_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column for their move using the minimax algorithm. """

        if self.maximizing: best_move_value = -9999
        else: best_move_value = 9999

        best_move = None

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        # Traverse all possible moces.
        for current_move in possible_moves:
            # Make the child node
            board_copy = copy.deepcopy(board)
            board_copy.board[current_move[0]][current_move[1]] = self.symbol

            # Evaluate the child node - the evaluation depends on the depth
            # The bigger the depth is the better the decision. 
            current_move_value = self.optimized_minimax(board = board_copy, depth=3, maximizing_player= not self.maximizing, alpha =-9999, beta = 9999, max_depth=3)[0] 
            
            # If it is a better move save it.
            if self.maximizing and current_move_value > best_move_value:
                best_move = current_move
                best_move_value = current_move_value
            elif not self.maximizing and current_move_value < best_move_value:
                best_move = current_move
                best_move_value = current_move_value

        return best_move[0], best_move[1]

    def _get_possible_moves(self, board_dimension, current_board):
        """ Privated method to get the possible moves where the board squares are empty. """

        moves = []
        # Traverse board to get empty squares
        for i in range(board_dimension):
            for j in range(board_dimension):
                if current_board[i][j] == '_':
                    moves.append((i,j))
        return moves

    def make_random_move(self, board_dimension, current_board):
        """ A method for the player to pick row and column at random for their move. """

        # Get possible moves: this slows the decision down but will only pick squares that are empty. 
        possible_moves = self._get_possible_moves(board_dimension, current_board)
        random_move = random.choice(possible_moves)

        return random_move[0], random_move[1]

    def minimax(self, board, depth, maximizing_player):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s

        Parameters
        ----------
        board: Board instance
            The current board configuration, child node in game tree
        depth: int
            The depth level in the game tree
        maximizing_player: Bool
            Indicates whether it is MAX or MIN's turn

        Returns
        -------
        value, steps: a tuple with evaluation of board configuration (child node) and number of child nodes explored
        """
        steps = 0

        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.h1(True, depth) != 0 or board.h1(False, depth) != 0 or board.is_board_full() == True:
            # I decided not to count this as a step and only the new children nodes that are explored.
            return  board.h1(self.maximizing, depth), steps
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                
                steps += 1 # Each valid child node (board configuration) explored is another step.
                
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'

                # Evaluate the child node
                result = self.minimax(board_copy, depth - 1, False)
                eval = result[0]
                steps += result[1]
                maxEval = max(maxEval, eval)

            return maxEval, steps

        else:
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999
            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:

                steps += 1 # Each valid child node (board configuration) explored is another step.
                
                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'

                # Evaluate the child node
                result = self.minimax(board_copy, depth - 1, True)
                eval = result[0]
                steps += result[1]
                minEval = min(minEval, eval)

            return minEval, steps

    def optimized_minimax(self, board, depth, alpha, beta, maximizing_player, max_depth):
        """ 
        A method for the bot player to make a decision based on the minimax algorithm with alpha beta pruning.
        
        I took the pseudocode from this video tutorial: https://www.youtube.com/watch?v=l-hh51ncgDI&t=314s

        Parameters
        ----------
        board: Board instance
            The current board configuration, child node in game tree
        depth: int
            The depth level in the game tree
        alpha: int
            Parameter with a value associated with the max nodes
        beta: int 
            Parameter with a value associated with the min nodes
        maximizing_player: Bool
            Indicates whether it is MAX or MIN's turn

        Returns
        -------
        value, steps: a tuple with evaluation of board configuration (child node) and number of child nodes explored
        """
        steps = 0
        
        # If depth for search (the level of the tree) is reached OR if the game is over (there's a winner) return static evaluation of the board. 
        if depth == 0 or board.h1(True, depth) != 0 or board.h1(False, depth) != 0 or board.is_board_full() == True:
            return board.h1(self.maximizing, max_depth-depth), steps
        
        if maximizing_player:
            # Simulate -infinity with a very big number. Can change with float string -inf but good enough. 
            maxEval = -9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:

                steps += 1 # Each valid child node (board configuration) explored is another step.

                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'X'

                # Evaluate the child node
                result = self.optimized_minimax(board_copy, depth - 1, alpha, beta, False, max_depth)
                eval = result[0]
                steps += result[1]

                maxEval = max(maxEval, eval)

                alpha = max(alpha, eval)
                if beta <= alpha:
                    break

            return maxEval, steps

        else:
            # Simulate infinity with a very small number. Can change with float string inf but good enough. 
            minEval = 9999

            possible_moves = self._get_possible_moves(board.board_demension, board.board)

            # Traverse all child nodes (board configurations) of current node (current board configuration)
            for move in possible_moves:
                
                steps += 1 # Each valid child node (board configuration) explored is another step.

                # Make the child node
                board_copy = copy.deepcopy(board)
                board_copy.board[move[0]][move[1]] = 'O'

                # Evaluate the child node
                result = self.optimized_minimax(board_copy, depth - 1, alpha, beta, True, max_depth)
                eval = result[0]
                steps += result[1]

                minEval = min(minEval, eval)

                beta = min(beta, eval)
                if beta <= alpha:
                    break

                
            return minEval, steps

{}


In [128]:
player1 = Player(human=False, symbol='X', maximizing=True)
board = Board(3)

board.board = [['_', 'O', 'X'],
                ['_', 'O', 'X'],
                ['X', 'X', '_']]

# Default case (current board - root node)
# assert(player1.optimized_minimax(board = board, depth=3, alpha = -9999, beta = 9999, maximizing_player=True, max_depth=3) == (99, 9))
print(player1.optimized_minimax(board = board, depth=3, alpha = -9999, beta = 9999, maximizing_player=True, max_depth=3))

# The maximizing player should win in a single move (2,2) rather than just playing the first move (0,0) that wins.
game = TicTacToeGame()
player2 = Player(human = True, maximizing = False, symbol = 'O')

game.play_game(player1 = player1, player2 = player2, board = board)

(99, 9)
Player X's turn
['_', 'O', 'X']
['_', 'O', 'X']
['X', 'X', '_']

🎉 Player X won! 🎉

['_', 'O', 'X']
['_', 'O', 'X']
['X', 'X', 'X']
